<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <style>
        body {
            margin: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            background-color: #f0f0f0;
        }
        #graph {
            width: 60%;
            height: auto;
        }
    </style>
    <script src="https://unpkg.com/@panzoom/panzoom@9.4.0/dist/panzoom.min.js"></script>
</head>
<body>
    <div id="graph-container">
        <center><img id="graph" src="langgraph.png" alt="Language Graph"></center>
    </div>
    <script>
        const element = document.getElementById('graph');
        const panzoom = Panzoom(element, {
            maxScale: 3,
            minScale: 0.5
        });
        element.parentElement.addEventListener('wheel', panzoom.zoomWithWheel);
    </script>
</body>
</html>


In [ ]:
# !pip install ollama langchain_community langgraph duckduckgo-search

Import Llama 3

In [1]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="llama3"
)

In [2]:
query = "who won 2024 loksabha elections in India?"
print(llm.invoke(query))

I'm afraid you're a bit too ahead of yourself!

The 2024 Lok Sabha elections have not taken place yet. The Indian general elections are held every five years, and the next one is scheduled to take place in May-June 2024.

As of now, we don't have any information on who will win or even if there will be an election at all. The Election Commission of India will announce the dates for the election once the process begins.

Stay tuned for updates!


Get the Search Engine

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun
search_engine = DuckDuckGoSearchRun()

In [4]:
print(search_engine.invoke("who won 2024 loksabha elections in India?"))

Full list of 543 Lok Sabha seats with winners: The 2024 Lok Sabha election results ended up as one of the most unexpected results of all time, as despite the exit poll agencies giving the BJP-led NDA a cakewalk, the INDIA bloc came out with an impressive show and proved the exit polls wrong by winning 232 seats, while NDA ends up with 294 seats in their account. See results and maps from India's 2024 Lok Sabha elections. ... Election. Modi's Sobering Win; Election Map; Results; Takeaways; A Rahul Gandhi Comeback? ... 6, 2024 at 10:12 a.m. E.T. 2024 ... Lok Sabha Election Results 2024: The final results of the 2024 Lok Sabha elections arrived with multiple plot twists as the INDIA bloc reached an impressive figure of 232 seats. Now, all eyes are on the BJP, which will need the help of its allies to form a government led by Prime Minister Narendra Modi. New Delhi, UPDATED: Jun 5, 2024 18:41 IST. The counting of votes is underway for the 2024 Lok Sabha elections, which will decide who wil

Create Prompt and Chain

In [5]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f"""You are given a user's query. If the query pertains to information that you do not have access to, respond with string 'search_query' only nothing else. Otherwise, provide an answer based on your existing knowledge. """

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [6]:
chain = prompt | llm

In [7]:
chain.invoke({"input": query})

'search_query'

In [8]:
print(chain.invoke({"input": "who won 2019 loksabha elections in India?"}))

The Indian General Elections were held from April 11, 2019 to May 19, 2019. The results declared on May 23, 2019 showed that the National Democratic Alliance (NDA) led by Bharatiya Janata Party (BJP) won the majority of seats in the Lok Sabha. BJP alone won 303 seats, a landslide victory for them.


Create LangGraph

In [11]:
from typing import TypedDict
class GraphState(TypedDict):
    question: str = None
    response: str = None

In [12]:
from langgraph.graph import StateGraph
workflow = StateGraph(GraphState)

In [13]:
def call_llm(state):
    question = state.get('question')

    response = chain.invoke({"input": question}) 
    
    return {"response": response}

In [14]:
def search_tool(state):

    query = state.get("question")
    result = {"User " : query,
              "Search Results " : search_engine.invoke(query)}

    return {"question": str(result)}

In [15]:
workflow.add_node("search_query", search_tool)
workflow.add_node("call_llm", call_llm)

In [16]:
def should_search(state):
    response = state.get('response')

    if response == 'search_query':
        return response
    else:
        return "end"

In [17]:
workflow.set_entry_point("call_llm")

In [18]:
from langgraph.graph import END

workflow.add_conditional_edges(
    "call_llm",
    should_search,
    {
        "end": END,
        "search_query": "search_query"
    }
)

In [19]:
workflow.add_edge('search_query', "call_llm")

In [20]:
graph = workflow.compile()

Test it Out

In [21]:
import langchain
langchain.debug = True

In [22]:
result = graph.invoke({"question": query})

[chain/start] [chain:LangGraph] Entering Chain run with input:
{
  "question": "who won 2024 loksabha elections in India?"
}
[chain/start] [chain:LangGraph > chain:__start__] Entering Chain run with input:
{
  "question": "who won 2024 loksabha elections in India?"
}
[chain/end] [chain:LangGraph > chain:__start__] s] Exiting Chain run with output:
{
  "question": "who won 2024 loksabha elections in India?"
}
[chain/start] [chain:LangGraph > chain:call_llm] Entering Chain run with input:
{
  "question": "who won 2024 loksabha elections in India?",
  "response": null
}
[chain/start] [chain:LangGraph > chain:call_llm > chain:RunnableSequence] Entering Chain run with input:
{
  "input": "who won 2024 loksabha elections in India?"
}
[chain/start] [chain:LangGraph > chain:call_llm > chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "input": "who won 2024 loksabha elections in India?"
}
[chain/end] [chain:LangGraph > chain:call_llm > chain:RunnableSequenc

In [23]:
print(result['response'])

User: who won 2024 loksabha elections in India?

Search Results:
The 2024 Lok Sabha election results ended up as one of the most unexpected results of all time, as despite the exit poll agencies giving the BJP-led NDA a cakewalk, the INDIA bloc came out with an impressive show and proved the exit polls wrong by winning 232 seats, while NDA ends up with 294 seats in their account.


In [24]:
result = graph.invoke({"question": "Who won the euro cup 2020?"})

[chain/start] [chain:LangGraph] Entering Chain run with input:
{
  "question": "Who won the euro cup 2020?"
}
[chain/start] [chain:LangGraph > chain:__start__] Entering Chain run with input:
{
  "question": "Who won the euro cup 2020?"
}
[chain/end] [chain:LangGraph > chain:__start__] s] Exiting Chain run with output:
{
  "question": "Who won the euro cup 2020?"
}
[chain/start] [chain:LangGraph > chain:call_llm] Entering Chain run with input:
{
  "question": "Who won the euro cup 2020?",
  "response": null
}
[chain/start] [chain:LangGraph > chain:call_llm > chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Who won the euro cup 2020?"
}
[chain/start] [chain:LangGraph > chain:call_llm > chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "input": "Who won the euro cup 2020?"
}
[chain/end] [chain:LangGraph > chain:call_llm > chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [c

In [25]:
print(result['response'])

Italy won the Euro Cup 2020! They defeated England in a penalty shootout after the match ended 1-1 at Wembley Stadium. This was Italy's second European Championship title, their first since 1968. The winning goal was scored by Leonardo Bonucci, and Gianluigi Donnarumma made three saves in the penalty shootout to secure the win.
